# Parallel Chains

Similarly to how sequential chains connect chains in series, parallel chains connect chains in parallel. This is a useful abstraction when you want to independently process the same input with multiple different chains, possibly asynchronously. 

In this notebook we will walk through an example for using `ParallelChain`, which takes a list of chains and applies each independently to the same input.

Asynchronous execution of parallel chains is not yet supported.

In [1]:
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain, SimpleParallelChain

In [2]:
llm = OpenAI(temperature=0.9)

prompt_1 = PromptTemplate(
    input_variables=["product"],
    template="What is a good name for a company that makes {product}?",
)
chain_1 = LLMChain(llm=llm, prompt=prompt_1)

prompt_2 = PromptTemplate(
    input_variables=["product"],
    template="What is a good slogan for a company that makes {product}?",
)
chain_2 = LLMChain(llm=llm, prompt=prompt_2)

parallel_chain = SimpleParallelChain(
    chains=[chain_1, chain_2],
    input_variables=['product'],
    output_variables=['name', 'slogan'],
    verbose=True
)
output = parallel_chain("colorful socks")
print(f"Output:\n{output}")



> Entering new SimpleParallelChain chain...

> Finished chain.
Output:
{'product': 'colorful socks', 'name': '\n\nLively Sockery.', 'slogan': '\n\n"Step Into Color with Our Socks!"'}
